In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files



import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim 
import folium # map rendering library
from geopy.exc import GeocoderTimedOut

from bs4 import BeautifulSoup
import requests
import urllib.request
import time


print('Libraries imported.')

Libraries imported.


Scrape top safest cities in Georgia into dataframe

In [2]:
gacrimeurl = 'https://www.alarms.org/safest-cities-in-georgia/'
gacrimeresp = requests.get(gacrimeurl)
#gacrimeresp
gacrimesoup = BeautifulSoup(gacrimeresp.text, 'lxml')
table = gacrimesoup.find('table', attrs={'class':'tablepress tablepress-id-101'})
table_rows = table.find_all('tr')
#create a list and append all text fields from each table row
ga_crime_list = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    ga_crime_list.append(row)
#convert list to dataframe    
ga_crime_df = pd.DataFrame(ga_crime_list, columns=['Rank', 'City', 'Violent Crimes', 'Property Crimes', 'Violent Crime Rate', 'Property Crime Rate'])
ga_crime_df = ga_crime_df.drop(ga_crime_df.index[0]) # drop first row as it came as junk
ga_crime_df['Rank'] = pd.to_numeric(ga_crime_df['Rank'])
ga_safe_20 = ga_crime_df.sort_values(by='Rank')[0:20]

In [3]:


ga_safe_20

,Rank,City,Violent Crimes,Property Crimes,Violent Crime Rate,Property Crime Rate
1,1,Holly Springs,12,144,1.030,12.357
2,2,Braselton,12,94,1.115,8.735
3,3,Milton,9,298,0.228,7.566
4,4,Johns Creek,25,575,0.294,6.761
5,5,Kennesaw,72,480,2.108,14.054
6,6,Duluth,33,557,1.108,18.694
7,7,Peachtree City,14,591,0.397,16.742
8,8,Roswell,108,"1,616",1.130,16.903
9,9,Grovetown,9,261,0.650,18.857
10,10,Bainbridge,47,379,3.850,31.045


Scapre Per Capita Income of cities in Georgia

In [4]:
ga_income_url = 'https://en.wikipedia.org/wiki/List_of_Georgia_(U.S._state)_locations_by_per_capita_income'
ga_income_resp = requests.get(ga_income_url)
ga_income_resp
ga_inc_soup = BeautifulSoup(ga_income_resp.text, 'lxml')
ga_inc_df = pd.DataFrame(columns=['City','Per Capita Income'])
ga_inc_soup = BeautifulSoup(ga_income_resp.text, 'lxml')
ga_inc_df = pd.DataFrame(columns=['City','Per Capita Income'])
for tag in ga_inc_soup.ol.find_all("li", recursive=True): 
    
    city = tag.text.split(',')[0]

    if("Cadwell" in city):
        percapita=tag.text.split('#')[1]
    else:
        percapita=tag.text.split('$')[1]
        
    ga_inc_df = ga_inc_df.append({'City' : city , 'Per Capita Income' : int(percapita.replace(',', ''))} , ignore_index=True)
    #print ('City is:' + city + ' percapita:' + percapita)
ga_inc_df['Per Capita Income'] = pd.to_numeric(ga_inc_df['Per Capita Income'])
ga_income_20 = ga_inc_df.sort_values(by=['Per Capita Income'], ascending=False)[0:20]

In [38]:
ga_income_20



,City,Per Capita Income
0,Berkeley Lake,69439
1,Johns Creek,65994
2,Dunwoody,62523
3,Vinings,61083
4,Chattahoochee Hills,52835
5,Milton,49628
6,Avondale Estates,42605
7,Alpharetta,42431
8,Woolsey,42177
9,Roswell,40106


In [37]:
atl_select = pd.merge(ga_inc_df,ga_crime_df, how='inner', on='City')
atl_select

,City,Per Capita Income,Rank,Violent Crimes,Property Crimes,Violent Crime Rate,Property Crime Rate
0,Johns Creek,65994,4,25,575,0.294,6.761
1,Dunwoody,62523,64,63,"2,003",1.277,40.612
2,Milton,49628,3,9,298,0.228,7.566
3,Roswell,40106,8,108,"1,616",1.130,16.903
4,Braselton,35921,2,12,94,1.115,8.735
5,Atlanta,35662,37,"4,504","22,991",9.357,47.764
6,Peachtree City,31667,7,14,591,0.397,16.742
7,Suwanee,29712,19,32,520,1.585,25.760
8,Decatur,29363,20,40,531,1.711,22.714
9,Duluth,29185,6,33,557,1.108,18.694


In [6]:
ga_select = pd.merge(ga_income_20,ga_safe_20, how='inner', on='City')
ga_select

,City,Per Capita Income,Rank,Violent Crimes,Property Crimes,Violent Crime Rate,Property Crime Rate
0,Johns Creek,65994,4,25,575,0.294,6.761
1,Milton,49628,3,9,298,0.228,7.566
2,Roswell,40106,8,108,"1,616",1.130,16.903
3,Braselton,35921,2,12,94,1.115,8.735
4,Peachtree City,31667,7,14,591,0.397,16.742


<b> Read the list of cities in Metro Atlanta area from csv file into a dataframe </b>

In [7]:
atlcities = pd.read_csv('atl_city_list.csv')

atlcities

,City,Latitude,Longitude
0,Acworth,NaN,NaN
1,Alpharetta,NaN,NaN
2,Atlanta,NaN,NaN
3,Auburn,NaN,NaN
4,Austell,NaN,NaN
5,Avondale Estates,NaN,NaN
6,Braselton,NaN,NaN
7,Buford,NaN,NaN
8,Canton,NaN,NaN
9,Chamblee,NaN,NaN


<b> Populate Latitude and Longitude values for each city using geolocator package </b>

In [8]:
def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [9]:
geolocator = Nominatim(user_agent="pn_explorer")
for index, row in atlcities.iterrows():
    address = '{}, Georgia'.format(row['City'])
    #location = geolocator.geocode(address)
    #print(address)
    location = do_geocode(address)
    atlcities.at[index, 'Latitude'] = location.latitude
    atlcities.at[index, 'Longitude'] = location.longitude


atlcities

,City,Latitude,Longitude
0,Acworth,34.065933,-84.676880
1,Alpharetta,34.070958,-84.274733
2,Atlanta,33.749099,-84.390185
3,Auburn,34.013662,-83.827350
4,Austell,33.812606,-84.634378
5,Avondale Estates,33.771494,-84.267144
6,Braselton,34.109274,-83.762673
7,Buford,34.120656,-84.004351
8,Canton,34.236762,-84.490762
9,Chamblee,33.892176,-84.298830


In [10]:
#atlcities.to_csv('atl_cities_lat_long.csv', index=False)


<b> Perform segmenting and clustering </b>

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim 
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [12]:
address = 'Atlanta, Georgia'

geolocator = Nominatim(user_agent="pn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Atlanta, Georgia are 33.7490987, -84.3901849.


Let's visualize Atlanta Metropolitan area with these cities

In [13]:
map_atl = folium.Map(location=[latitude, longitude], zoom_start=9)
from IPython.display import display

# add markers to map
for lat, lng, label in zip(atlcities['Latitude'], atlcities['Longitude'], atlcities['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_atl)  
    
map_atl

Define Foursquare Credentials and Version

In [14]:
CLIENT_ID = '30ZMHA3YFQZNVXOXBXOJEV3GF14DJ42V05NNJUQR4E0XBNAP' # your Foursquare ID
CLIENT_SECRET = 'TNQO4VSBXW2BCURPAESJ4OGENGKR20KZPM10X5L2T35ZNFGU' # your Foursquare Secret
VERSION = '20191003' # Foursquare API version


Let's find out the top 100 venues near Sandy Springs

In [15]:
ss_lat = atlcities.loc[atlcities['City']=='Sandy Springs'].Latitude.item()
ss_long = atlcities.loc[atlcities['City']=='Sandy Springs'].Longitude.item()

radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    ss_lat, 
    ss_long, 
    radius, 
    LIMIT)
results = requests.get(url).json()
results
                                                               

{'meta': {'code': 200, 'requestId': '5d990a43ad1789002de5064a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Downtown Sandy Springs',
  'headerFullLocation': 'Downtown Sandy Springs, Sandy Springs',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 61,
  'suggestedBounds': {'ne': {'lat': 33.9287688045, 'lng': -84.37312486966512},
   'sw': {'lat': 33.9197687955, 'lng': -84.38395093033488}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a84b175f964a5205afd1fe3',
       'name': "Trader Joe's",
       'location': {'address': '6277 Roswell Road NE',
        'crossStreet': 'at Sandy Springs Cir NE',
        'lat': 33.926247668

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.head())
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

                                   name                 categories        lat  \
0                          Trader Joe's              Grocery Store  33.926248   
1  Sandy Springs Performing Arts Center      Performing Arts Venue  33.924610   
2                        Rumi's Kitchen  Middle Eastern Restaurant  33.922002   
3                              Roasters        American Restaurant  33.925188   
4                             Five Guys               Burger Joint  33.926733   

         lng  
0 -84.378479  
1 -84.379760  
2 -84.379551  
3 -84.378368  
4 -84.378496  
61 venues were returned by Foursquare.


Now repeat this process to the neighborhoods in our final dataset

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
atl_venues = getNearbyVenues(names=atlcities['City'],
                                   latitudes=atlcities['Latitude'],
                                   longitudes=atlcities['Longitude']
                                  )

Acworth
Alpharetta
Atlanta
Auburn
Austell
Avondale Estates
Braselton
Buford
Canton
Chamblee
Clarkston
College Park
Conyers
Covington
Cumming
Dacula
Decatur
Doraville
Douglasville
Duluth
Dunwoody
East Point
Ellenwood
Fairburn
Fayetteville
Flowery Branch
Forest Park
Grayson
Hampton
Hapeville
Hiram
Johns Creek
Jonesboro
Kennesaw
Lawrenceville
Lilburn
Lithia Springs
Lithonia
Locust Grove
Loganville
Mableton
Marietta
McDonough
Milton
Morrow
Newnan
Norcross
Peachtree City
Powder Springs
Riverdale
Roswell
Sandy Springs
Scottdale
Senoia
Smyrna
Snellville
Stockbridge
Stone Mountain
Sunny Side
Suwanee
Tucker
Union City
Vinings
West Oak
Woodstock


In [20]:
print(atl_venues.shape)
atl_venues.head()



(1072, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acworth,34.065933,-84.67688,Henry's Louisiana Grill,34.066011,-84.677728,Cajun / Creole Restaurant
1,Acworth,34.065933,-84.67688,Fusco's via Roma,34.065781,-84.677163,Italian Restaurant
2,Acworth,34.065933,-84.67688,Miss L's Sandwich Shop,34.065861,-84.677276,Sandwich Place
3,Acworth,34.065933,-84.67688,Lacey's Drugs,34.065340,-84.676674,Pharmacy
4,Acworth,34.065933,-84.67688,Oak Barrel,34.066145,-84.678016,Wine Shop


Let's check how many venues were returned for each neighborhood

In [21]:
atl_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Acworth,14,14,14,14,14,14
Alpharetta,46,46,46,46,46,46
Atlanta,10,10,10,10,10,10
Auburn,5,5,5,5,5,5
Austell,7,7,7,7,7,7
Avondale Estates,5,5,5,5,5,5
Braselton,8,8,8,8,8,8
Buford,10,10,10,10,10,10
Canton,12,12,12,12,12,12


In [22]:
print('There are {} unique categories.'.format(len(atl_venues['Venue Category'].unique())))

There are 216 unique categories.


Analyze Each PostalCode 

In [23]:
# one hot encoding
atl_onehot = pd.get_dummies(atl_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
atl_onehot['City'] = atl_venues['City'] 

# move city column to the first column
fixed_columns = [atl_onehot.columns[43]] + list(atl_onehot.columns[:43]) + list(atl_onehot.columns[44:])
atl_onehot = atl_onehot[fixed_columns]

atl_onehot.head()


,City,Accessories Store,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Martial Arts Dojo,Massage Studio,Medical Center,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Office,Optical Shop,Other Great Outdoors,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Acworth,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Acworth,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Acworth,0,0,0,0,0,0,0

Next, let's group rows by city and by taking the mean of the frequency of occurrence of each category

In [24]:
atl_grouped = atl_onehot.groupby('City').mean().reset_index()
atl_grouped

,City,Accessories Store,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Martial Arts Dojo,Massage Studio,Medical Center,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Office,Optical Shop,Other Great Outdoors,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Acworth,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.071429,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [25]:
atl_grouped.shape

(65, 216)

Let's print each neighborhood along with the top 5 most common venues

In [26]:
num_top_venues = 5

for city in atl_grouped['City']:
    print("----"+city+"----")
    temp = atl_grouped[atl_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acworth----
                       venue  freq
0        American Restaurant  0.14
1         Italian Restaurant  0.07
2                     Bakery  0.07
3  Cajun / Creole Restaurant  0.07
4    New American Restaurant  0.07


----Alpharetta----
                     venue  freq
0           Clothing Store  0.13
1   Furniture / Home Store  0.09
2  New American Restaurant  0.07
3         Sushi Restaurant  0.04
4      American Restaurant  0.04


----Atlanta----
                 venue  freq
0          Event Space   0.1
1         Intersection   0.1
2  American Restaurant   0.1
3          Art Gallery   0.1
4          Music Venue   0.1


----Auburn----
                venue  freq
0           City Hall   0.2
1   Convenience Store   0.2
2           BBQ Joint   0.2
3  Chinese Restaurant   0.2
4      Baseball Field   0.2


----Austell----
                venue  freq
0  Mexican Restaurant  0.29
1               Diner  0.14
2            Pharmacy  0.14
3           Pet Store  0.14
4           Gift Sho

               venue  freq
0   Business Service  0.67
1  Recreation Center  0.33
2  Accessories Store  0.00
3    Paintball Field  0.00
4      Movie Theater  0.00


----Morrow----
                   venue  freq
0  Vietnamese Restaurant  0.19
1   Fast Food Restaurant  0.19
2                  Hotel  0.06
3         Breakfast Spot  0.06
4   Caribbean Restaurant  0.06


----Newnan----
                  venue  freq
0        Discount Store   0.2
1             BBQ Joint   0.2
2                  Food   0.2
3            Steakhouse   0.2
4  Fast Food Restaurant   0.2


----Norcross----
                venue  freq
0                Park  0.13
1  Italian Restaurant  0.07
2              Bakery  0.07
3    Cuban Restaurant  0.07
4         Pizza Place  0.07


----Peachtree City----
                 venue  freq
0       Sandwich Place  0.05
1       Lingerie Store  0.03
2  American Restaurant  0.03
3       Clothing Store  0.03
4    Electronics Store  0.03


----Powder Springs----
                    venue  

In [27]:
#First, let's write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each city

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cities_venues_sorted = pd.DataFrame(columns=columns)
cities_venues_sorted['City'] = atl_grouped['City']

for ind in np.arange(atl_grouped.shape[0]):
    cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(atl_grouped.iloc[ind, :], num_top_venues)

cities_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acworth,American Restaurant,Sporting Goods Shop,Bakery,Pharmacy,Wine Shop,Food Court,Bike Shop,New American Restaurant,Italian Restaurant,Health & Beauty Service
1,Alpharetta,Clothing Store,Furniture / Home Store,New American Restaurant,Kids Store,Women's Store,Sushi Restaurant,American Restaurant,Plaza,Coffee Shop,Pizza Place
2,Atlanta,Event Space,Rental Car Location,American Restaurant,Caribbean Restaurant,Intersection,Music Venue,Art Gallery,Breakfast Spot,Food Court,College Basketball Court
3,Auburn,Convenience Store,BBQ Joint,Baseball Field,City Hall,Chinese Restaurant,Dessert Shop,Event Space,French Restaurant,Football Stadium,Food Truck
4,Austell,Mexican Restaurant,Gift Shop,Pharmacy,Pet Store,Park,Diner,Yoga Studio,Food Truck,Food Court,Food & Drink Shop


Cluster the cities

Run k-means to cluster the cities into 5 clusters

In [29]:
# set number of clusters
kclusters = 5

atl_grouped_clustering = atl_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(atl_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_


array([0, 0, 0, 0, 4, 4, 4, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 4, 0, 4, 0, 4, 0, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 1,
       0, 3, 4, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 2, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each city.

In [30]:
# add clustering labels
cities_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
atl_merged = atlcities
# merge atl_grouped with atl_data to add latitude/longitude for each city
atl_merged = atl_merged.join(cities_venues_sorted.set_index('City'), on='City')

atl_merged.head() # check the last columns!



,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acworth,34.065933,-84.676880,0,American Restaurant,Sporting Goods Shop,Bakery,Pharmacy,Wine Shop,Food Court,Bike Shop,New American Restaurant,Italian Restaurant,Health & Beauty Service
1,Alpharetta,34.070958,-84.274733,0,Clothing Store,Furniture / Home Store,New American Restaurant,Kids Store,Women's Store,Sushi Restaurant,American Restaurant,Plaza,Coffee Shop,Pizza Place
2,Atlanta,33.749099,-84.390185,0,Event Space,Rental Car Location,American Restaurant,Caribbean Restaurant,Intersection,Music Venue,Art Gallery,Breakfast Spot,Food Court,College Basketball Court
3,Auburn,34.013662,-83.827350,0,Convenience Store,BBQ Joint,Baseball Field,City Hall,Chinese Restaurant,Dessert Shop,Event Space,French Restaurant,Football Stadium,Food Truck
4,Austell,33.812606,-84.634378,4,Mexican Restaurant,Gift Shop,Pharmacy,Pet Store,Park,Diner,Yoga Studio,Food Truck,Food Court,Food & Drink Shop


Finally, let's visualize the resulting clusters

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(atl_merged['Latitude'], atl_merged['Longitude'], atl_merged['City'], atl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine the 5 clusters now

In [32]:
atl_merged.loc[atl_merged['Cluster Labels'] == 0, atl_merged.columns[[0] + list(range(4,atl_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acworth,American Restaurant,Sporting Goods Shop,Bakery,Pharmacy,Wine Shop,Food Court,Bike Shop,New American Restaurant,Italian Restaurant,Health & Beauty Service
1,Alpharetta,Clothing Store,Furniture / Home Store,New American Restaurant,Kids Store,Women's Store,Sushi Restaurant,American Restaurant,Plaza,Coffee Shop,Pizza Place
2,Atlanta,Event Space,Rental Car Location,American Restaurant,Caribbean Restaurant,Intersection,Music Venue,Art Gallery,Breakfast Spot,Food Court,College Basketball Court
3,Auburn,Convenience Store,BBQ Joint,Baseball Field,City Hall,Chinese Restaurant,Dessert Shop,Event Space,French Restaurant,Football Stadium,Food Truck
7,Buford,Insurance Office,Bistro,New American Restaurant,Bar,Tapas Restaurant,Discount Store,Steakhouse,Italian Restaurant,Hookah Bar,American Restaurant
8,Canton,American Restaurant,Playground,Jewelry Store,Soccer Field,Bookstore,Gastropub,Seafood Restaurant,Sandwich Place,Diner,Southern / Soul Food Restaurant
9,Chamblee,Chinese Restaurant,Dim Sum Restaurant,Asian Restaurant,Furniture / Home Store,Indian Restaurant,Grocery Store,Ice Cream Shop,Restaurant,Shopping Mall,Fried Chicken Joint
11,College Park,American Restaurant,Restaurant,Athletics & Sports,Metro Station,Football Stadium,Business Service,Shoe Store,Seafood Restaurant,BBQ Joint,Southern / Soul Food Restaurant
12,Conyers,Bar,Plaza,Jewelry Store,New American Restaurant,Thai Restaurant,Greek Restaurant,Ice Cream Shop,Mexican Restaurant,Pub,Ethiopian Restaurant
13,Covington,Ice Cream Shop,BBQ Joint,Deli / Bodega,Cajun / Creole Restaurant,Southern / Soul Food Restaurant,Pizza Place,Gas Station,Tourist Information Center,Bakery,Plaza


In [33]:
atl_merged.loc[atl_merged['Cluster Labels'] == 1, atl_merged.columns[[0] + list(range(4,atl_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Milton,Business Service,Recreation Center,Yoga Studio,Electronics Store,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop


In [34]:
atl_merged.loc[atl_merged['Cluster Labels'] == 2, atl_merged.columns[[0] + list(range(4,atl_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,West Oak,Food,Gym / Fitness Center,Yoga Studio,Electronics Store,French Restaurant,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Flower Shop


In [35]:
atl_merged.loc[atl_merged['Cluster Labels'] == 3, atl_merged.columns[[0] + list(range(4,atl_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Clarkston,Discount Store,Home Service,Indian Restaurant,Grocery Store,Photography Studio,Middle Eastern Restaurant,Deli / Bodega,Event Space,Cupcake Shop,Football Stadium
36,Lithia Springs,Discount Store,Pharmacy,Moving Target,Video Store,Gym / Fitness Center,Locksmith,Electronics Store,Food Truck,Food Court,Food & Drink Shop
40,Mableton,Fast Food Restaurant,Discount Store,Bookstore,Fried Chicken Joint,Food,Gym,Gas Station,Mexican Restaurant,Arts & Crafts Store,Boutique
45,Newnan,Steakhouse,Food,Fast Food Restaurant,BBQ Joint,Discount Store,Yoga Studio,Ethiopian Restaurant,French Restaurant,Football Stadium,Food Truck
49,Riverdale,Fast Food Restaurant,Pharmacy,Liquor Store,Wings Joint,Office,Rental Car Location,Discount Store,Donut Shop,Sandwich Place,Mexican Restaurant


In [36]:
atl_merged.loc[atl_merged['Cluster Labels'] == 4, atl_merged.columns[[0] + list(range(4,atl_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Austell,Mexican Restaurant,Gift Shop,Pharmacy,Pet Store,Park,Diner,Yoga Studio,Food Truck,Food Court,Food & Drink Shop
5,Avondale Estates,Pool,Playground,Business Service,Park,Electronics Store,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Food
6,Braselton,Gift Shop,Locksmith,American Restaurant,New American Restaurant,Mexican Restaurant,Home Service,Steakhouse,Park,Event Space,Farmers Market
26,Forest Park,Insurance Office,Playground,Park,Paper / Office Supplies Store,Electronics Store,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Food
28,Hampton,Brewery,Wings Joint,Mexican Restaurant,Bookstore,Pharmacy,Park,Food & Drink Shop,Ethiopian Restaurant,Football Stadium,Food Truck
30,Hiram,Park,Greek Restaurant,American Restaurant,Dance Studio,Business Service,Coffee Shop,Smoke Shop,Music Store,Event Space,Food Truck
46,Norcross,Park,Construction & Landscaping,Bakery,Cuban Restaurant,Comfort Food Restaurant,Pub,Coffee Shop,Mexican Restaurant,Steakhouse,Gift Shop
57,Stone Mountain,Mountain,Park,Campground,Yoga Studio,Ethiopian Restaurant,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Food
58,Sunny Side,Gas Station,Park,Baseball Field,Yoga Studio,Event Space,French Restaurant,Football Stadium,Food Truck,Food Court,Food & Drink Shop
